In [ ]:
use role ACCOUNTADMIN;

In [ ]:
create or replace database HUD_DEMO;
CREATE STAGE docstage
  ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE')
  DIRECTORY=(ENABLE=true);

In [ ]:
drop database if exists HUD_HOUSINGFORECASTBYZIPCODE;

### Access Snowflake Marketplace and search for the listing:  ***US Housing Forecast by Zip Code | 2010 - 2035***
### "Get" the listing and name the database "HUD_HOUSINGFORECASTBYZIPCODE"

In [ ]:
use role ACCOUNTADMIN;
use database HUD_HOUSINGFORECASTBYZIPCODE;
use schema ATERIO_DATASHEET_DEV;

### This demo uses some of Snowflake's latest innovations to illustrate how easily data (both structured and unstructured) can be assembled from external sources and used to provide insights by state-of-the-art agentic tools.

Specific technologies that will be highlighed include:

- Snowflake Intelligence
- Cortex Agents
- Cortex Search (with PARSE_DOCUMENT and SPLIT_TEXT_RECURSIVE_CHARACTER functions)
- Cortex Analyst
- Semantic Views
- Snowflake Marketplace

In [ ]:
import streamlit as st
st.image("HUDDemoArch.png")

### Look at the data we accessed thru Marketplace

In [ ]:
select * from US_HOUSING_FORECAST_ZIPCODE limit 10;

In [ ]:
use schema HUD_DEMO.PUBLIC;

### To create the Semantic View, we want a copy of the source table in a writable schema

In [ ]:
create or replace table US_HOUSING_FORECAST_ZIPCODE as 
select * from HUD_HOUSINGFORECASTBYZIPCODE.ATERIO_DATASHEET_DEV.US_HOUSING_FORECAST_ZIPCODE;

### Build the RAG pipeline for unstructured documents

In [ ]:
create or replace TABLE HUD_DEMO.PUBLIC.DOC_CHUNKS_TABLE (
	FILENAME VARCHAR(16777216),
	SIZE NUMBER(38,0),
	FILE_URL VARCHAR(16777216),
	CHUNK VARCHAR(16777216)
);

In [ ]:
create or replace CORTEX SEARCH SERVICE GPRag_Service
ON chunk

warehouse = COMPUTE_WH
TARGET_LAG = '1 minute'
as (
    select chunk,
        filename,
        file_url
    from DOC_CHUNKS_TABLE
);

In [ ]:
create or replace task process_new_files 
    warehouse = COMPUTE_WH
    schedule = '1 minutes'
    when system$stream_has_data('new_doc_stream')
    as
        BEGIN
            CREATE OR REPLACE TEMPORARY TABLE PARSED_DOCUMENTS AS
            SELECT 
                RELATIVE_PATH AS FILENAME,
                FILE_URL,
                SIZE,
                LAST_MODIFIED,
                TO_VARCHAR (
                    SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
                        '@DocStage',
                         relative_path,
                         {'mode': 'layout'}
                    )
                ) AS PARSED_CONTENT
            FROM 
                DIRECTORY(@DocStage) 
            WHERE FILENAME IN (SELECT RELATIVE_PATH FROM new_doc_stream); -- for new documents only
                
            INSERT INTO DOC_CHUNKS_TABLE
               select filename, 
                    size,
                    file_url, 
                    to_varchar(c.value) as chunk 
               from PARSED_DOCUMENTS,
                   LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
                   PARSED_CONTENT,
                  'markdown',
                  1512, -- Chunk size
                  256 -- Overlap
               )) c;
                
        END;


create or replace stream new_doc_stream on stage DocStage;
alter task process_new_files  resume;

### Download the PDF from the HUD website here: https://www.huduser.gov/portal/publications/pdf/National-CHMA-24.pdf

### Then upload to docstage in HUD_DEMO.PUBLIC


### Wait a couple minutes for the task to chunk it up and add to the Cortex Search service

In [ ]:
select * from DIRECTORY(@DocStage) ;

In [ ]:
select * from DOC_CHUNKS_TABLE limit 10;

In [ ]:
SELECT
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW (
      'GPRag_Service',
      '{
          "query": "What are the major factors impacting the US housing market in 2024 ?",
          "columns": ["FILENAME", "chunk"],
          "limit": 3
      }'
  ) as cs_context
  LIMIT 3;

In [ ]:
select SNOWFLAKE.CORTEX.COMPLETE('mistral-large',concat('What are the major factors impacting the US housing market in 2024',cs_context)) 
FROM {{CortexSearchPreview}};

### Now, lets create a Semantic View so we can use Cortex Analyst against the data set we got from Marketplace.  We will do this through the UI (AI & ML -> Cortex Analyst), but the object that gets created looks like this: 

In [ ]:
create or replace semantic view HUD_DEMO.PUBLIC.HUD_US_HOUSING_FORECAST_BY_ZIPCODE
	tables (
		US_HOUSING_FORECAST_ZIPCODE comment='This table stores historical and forecasted data on the US housing market at the zip code level, including metrics such as total homes available, building permits, average people per home, estimated total home demand, and vacancy rates, as well as demographic information like owner-occupied and renter-occupied housing units, to provide insights into the supply and demand dynamics of the US housing market.'
	)
	facts (
		US_HOUSING_FORECAST_ZIPCODE.AVG_PEOPLE_PER_HOME as AVG_PEOPLE_PER_HOME comment='The average number of people residing in a single-family home within a specific zip code area.',
		US_HOUSING_FORECAST_ZIPCODE.EST_TOTAL_HOME_DEMAND_2030 as EST_TOTAL_HOME_DEMAND_2030 comment='Estimated total home demand in the year 2030 for a specific zip code area.',
		US_HOUSING_FORECAST_ZIPCODE.IDX_DEMAND_SUPPLY as IDX_DEMAND_SUPPLY comment='The IDX_DEMAND_SUPPLY column represents the ratio of demand to supply in the housing market for a specific zip code, indicating the balance between the number of potential buyers and the number of available homes for sale.',
		US_HOUSING_FORECAST_ZIPCODE.RT_BUILDING_PERMIT_YOY_2022_2021 as RT_BUILDING_PERMIT_YOY_2022_2021 comment='Year-over-year percentage change in building permits issued in 2022 compared to 2021.',
		US_HOUSING_FORECAST_ZIPCODE.RT_VACANCY as RT_VACANCY comment='The percentage of vacant homes in a given zip code area.',
		US_HOUSING_FORECAST_ZIPCODE.RT_VACANCY_RENTAL as RT_VACANCY_RENTAL comment='The percentage of rental properties in a given zip code that are currently vacant and available for rent.',
		US_HOUSING_FORECAST_ZIPCODE.TOT_BALANCE_INVENTORY as TOT_BALANCE_INVENTORY comment='Total balance of housing inventory, representing the total value of unsold homes in a given zip code, with negative values indicating a shortage of inventory and positive values indicating a surplus.'
	)
	dimensions (
		US_HOUSING_FORECAST_ZIPCODE.CITY_NAME as CITY_NAME comment='The name of the city within the United States where the housing forecast data is being tracked.',
		US_HOUSING_FORECAST_ZIPCODE.COUNTY_FIPS_CODE as COUNTY_FIPS_CODE comment='A unique identifier for the county in which the zip code is located, as assigned by the US Census Bureau''s Federal Information Processing Standard (FIPS) publication.',
		US_HOUSING_FORECAST_ZIPCODE.COUNTY_NAME as COUNTY_NAME comment='The county in which the zip code is located.',
		US_HOUSING_FORECAST_ZIPCODE.LBL_LATEST_HOUSING_YEAR_DATA as LBL_LATEST_HOUSING_YEAR_DATA comment='The year for which the latest housing data is available for a specific zip code.',
		US_HOUSING_FORECAST_ZIPCODE.STATE_CODE as STATE_CODE comment='The two-character code representing the state in the United States where the zip code is located.',
		US_HOUSING_FORECAST_ZIPCODE.TOT_FX_CONSTRUCTION_5Y as TOT_FX_CONSTRUCTION_5Y comment='Total forecasted construction value for the next 5 years.',
		US_HOUSING_FORECAST_ZIPCODE.TOT_HOME_AVAILABLE_2020 as TOT_HOME_AVAILABLE_2020 comment='Total number of homes available in a specific zip code area as of 2020.',
		US_HOUSING_FORECAST_ZIPCODE.TOT_HOME_AVAILABLE_2021 as TOT_HOME_AVAILABLE_2021 comment='Total number of homes available in a specific zip code area as of 2021.',
		US_HOUSING_FORECAST_ZIPCODE.TOT_HOUSING_UNIT as TOT_HOUSING_UNIT comment='Total number of housing units in a given zip code area.',
		US_HOUSING_FORECAST_ZIPCODE.TOT_OWNER_OCCUPIED_HOUSING_UNIT as TOT_OWNER_OCCUPIED_HOUSING_UNIT comment='Total number of housing units that are occupied by their owners in a given zip code.',
		US_HOUSING_FORECAST_ZIPCODE.TOT_RENTER_OCCUPIED_HOUSING_UNIT as TOT_RENTER_OCCUPIED_HOUSING_UNIT comment='Total number of renter-occupied housing units in a given zip code.',
		US_HOUSING_FORECAST_ZIPCODE.UPDATED_AT as UPDATED_AT comment='The date and time when the housing forecast data for the specific zipcode was last updated.',
		US_HOUSING_FORECAST_ZIPCODE.ZIP_CODE as ZIP_CODE comment='The five-digit code that identifies a specific geographic area in the United States, used to organize and route mail, and also serves as a unique identifier for analyzing housing market trends and forecasts at a local level.'
	)
	comment='This view has housing data for Oregon and Washington'
	with extension (CA='{"tables":[{"name":"US_HOUSING_FORECAST_ZIPCODE","dimensions":[{"name":"CITY_NAME","sample_values":["Edwall","Springfield","Bridal Veil"]},{"name":"COUNTY_FIPS_CODE","sample_values":["41035","53053","53043"]},{"name":"COUNTY_NAME","sample_values":["Jackson County","Multnomah County","Klamath County"]},{"name":"LBL_LATEST_HOUSING_YEAR_DATA","sample_values":["2021"]},{"name":"STATE_CODE","sample_values":["WA","OR"]},{"name":"TOT_FX_CONSTRUCTION_5Y"},{"name":"TOT_HOME_AVAILABLE_2020","sample_values":["393","420","13320"]},{"name":"TOT_HOME_AVAILABLE_2021","sample_values":["2414","2122","1370"]},{"name":"TOT_HOUSING_UNIT","sample_values":["2414","2122","1370"]},{"name":"TOT_OWNER_OCCUPIED_HOUSING_UNIT","sample_values":["5455","7471","428"]},{"name":"TOT_RENTER_OCCUPIED_HOUSING_UNIT","sample_values":["934","1383","4729"]},{"name":"ZIP_CODE","sample_values":["98354","97520","97357"]}],"facts":[{"name":"AVG_PEOPLE_PER_HOME","sample_values":["3.186047","2.484316","2.062500"]},{"name":"EST_TOTAL_HOME_DEMAND_2030","sample_values":["138.801289","14477.253784","111.797927"]},{"name":"IDX_DEMAND_SUPPLY","sample_values":["42.15070390","30.75087382","74.50038856"]},{"name":"RT_BUILDING_PERMIT_YOY_2022_2021","sample_values":["-0.27595884","0.13333333","-0.32957111"]},{"name":"RT_VACANCY","sample_values":["0.04000600","0.18290400","0.09784200"]},{"name":"RT_VACANCY_RENTAL","sample_values":["0.08600000","0.07400000","0.02500000"]},{"name":"TOT_BALANCE_INVENTORY","sample_values":["-1.569349","87.486628","755.152345"]}],"time_dimensions":[{"name":"UPDATED_AT","sample_values":["2024-02-27T20:23:28.638+0000"]}]}]}');

We now have two defined "tools" that will be defined to an Agent that we create.  These are visible in HUD_DEMO.PUBLIC

1. HUD_US_HOUSING_FORECAST_BY_ZIPCODE (Semantic View used by Cortex Analyst)
2. GPRAG_SERVICE (Cortex Search Service)

1. Click **AI & ML -> Agents**
2. Click **Create Agent**. Call the new agent **US_HUD_AGENT**.  Check the box labeled "Create this agent for
Snowflake Intelligence"
3. Add a description ("Handy agent allowing you to ask questions of HUD data and documents")
4. Click the Tools tab 
- Add a Cortex Analyst service
- Add a Cortex Search service 